In [1]:
!pip install pytorch-lightning --quiet
!pip install transformers --quiet
!pip install datasets --quiet
!pip install onnx --quiet
!pip install onnxruntime --quiet
!pip install optimum --quiet

     |████████████████████████████████| 585 kB 36.1 MB/s 
     |████████████████████████████████| 419 kB 54.7 MB/s 
     |████████████████████████████████| 141 kB 55.8 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 4.7 MB 35.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.3 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 365 kB 36.0 MB/s 
     |████████████████████████████████| 212 kB 76.2 MB/s 
     |████████████████████████████████| 127 kB 71.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 13.1 MB 29.4 MB/s 
     |████████████████████████████████| 4.9 MB 34.2 MB/s 
     |██████████████████████████████

In [ ]:
# read data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
import pandas as pd
import numpy as np
df1=pd.read_excel('/content/Covid humour data.xlsx',0)
df2=pd.read_excel('/content/Covid humour data.xlsx',1)
len(df1),len(df2)


(1384, 3722)

In [16]:
df=pd.concat([df1.iloc[:,0],df2.iloc[:,0]],axis=0)
df=df.reset_index()
df['label']=pd.Series([0]*len(df1)+[1]*len(df2))
df.drop('index',axis=1,inplace=True)
df.columns=['tweet','label']
df.head()

,tweet,label
0,"We can look at the damage made by “COVID-19”, ...",0
1,What's got 400 legs and 7 teeth?... The queue ...,0
2,Chuck Norris got the Coronavirus. The virus is...,0
3,I heard a new term today: when were you born? ...,0
4,Why did the Coronavirus cross the street?... ....,0


In [17]:
def clean(tweets):
    tweets.tweet=tweets.tweet.str.replace(r'@user', '', regex=True)
    tweets.tweet=tweets.tweet.str.lower()
    tweets.tweet=tweets.tweet.str.replace(r'[^\w\s]+', '')
    tweets.tweet=tweets.tweet.str.replace(r'\s\s+', '')
    tweets.tweet=tweets.tweet.str.strip()
    return tweets
df=clean(df)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df.duplicated().sum()

0

In [20]:
df.tail()

,tweet,label
5101,35 new cases and 2 new deaths in somalia2106 g...,1
5102,9474 new cases and 136 new deaths in peru2101 ...,1
5103,breaking india added nearly 79000 fresh cases ...,1
5104,63 new cases and 1 new death in mozambique2054...,1
5105,210 new cases and 4 new deaths in namibia2048 ...,1


In [21]:
df.label.value_counts()

1    3697
0    1170
Name: label, dtype: int64

In [22]:
g = df.groupby('label')
df=g.apply(lambda x: x.sample(g.size().min(),random_state=1).reset_index(drop=True)).reset_index(drop=True)

In [23]:
from sklearn.model_selection import train_test_split
train_df,val_df=train_test_split(df,test_size=0.2,random_state=0)
train_df.reset_index(inplace=True,drop=True)
val_df.reset_index(inplace=True,drop=True)
len(train_df),len(val_df)

(1872, 468)

In [24]:
from pytorch_lightning import seed_everything, LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping
from torch.nn.functional import cross_entropy
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import torchmetrics
from sklearn.metrics import classification_report,precision_recall_fscore_support,accuracy_score
import torch#pytorch
from transformers import AutoModelForSequenceClassification


In [25]:
model_name="distilbert-base-uncased-finetuned-sst-2-english"
from transformers import AutoTokenizer, AutoModel#for embeddings
# create data loader 
from torch.utils.data import DataLoader,Dataset
class DataReader(Dataset):
  def __init__(self,dataframe,test=False):
    super(DataReader,self).__init__()
   # self.tokenizer=tokenizer
    self.df=dataframe
    self.test=test
    self.tokenizer = AutoTokenizer.from_pretrained(model_name,local_files_only=False)

  def __len__(self):
    return len(self.df)

  def __getitem__(self,index):
    text=self.df.tweet[index]
    tokens=self.tokenizer(text,max_length=64,padding='max_length',truncation=True,return_tensors='pt')
    if self.test:#no labels
      return tokens.input_ids,tokens.attention_mask
    else:
      label=self.df.label[index]
      return tokens.input_ids,tokens.attention_mask,label



In [26]:
class OurModel(LightningModule):
    def __init__(self ):
        super(OurModel,self).__init__()
      
  
        #pretrained model
        self.bertmodel=AutoModelForSequenceClassification.from_pretrained(model_name)
        #parameters
        self.lr =5e-5
        self.batch_size=4
        self.numworker=2
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
         
        self.criterion=nn.CrossEntropyLoss()
        self.ground_label,self.predicted_label=0,0
        self.report=[]
    def forward(self,input_ids,attention_mask):
        out=self.bertmodel(input_ids.squeeze(1),attention_mask.squeeze(1)).logits
        return out
    

    
    def configure_optimizers(self):
        return torch.optim.AdamW(params=self.parameters(),lr=self.lr )

    def train_dataloader(self):
        return DataLoader(DataReader(train_df), batch_size = self.batch_size,
                          num_workers=self.numworker,pin_memory=True, shuffle=False,
                         
                         )

    def training_step(self,batch,batch_idx):
        input_ids,attention_mask,label=batch
        out=self.forward(input_ids,attention_mask)
        loss=self.criterion(out,label)
        acc=self.train_acc(out,label)
        return {'loss':loss,'acc':acc}
    
    def training_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.log('train_loss', loss)
        self.log('train_acc',acc)
             
    def val_dataloader(self):
        return DataLoader(DataReader(val_df), batch_size = self.batch_size,
                          num_workers=self.numworker,pin_memory=True,  shuffle=False)
    
    def validation_step(self,batch,batch_idx):
        input_ids,attention_mask,label=batch
        out=self(input_ids,attention_mask)
        loss=self.criterion(out,label)
        acc=self.train_acc(out,label)
        return {'loss':loss,'acc':acc,'pred':out,'label':label}
    
    def validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        print('validation accuracy',acc)
        
        self.log('val_loss', loss)
        self.log('val_acc',acc)

        
    def test_dataloader(self):
        return DataLoader(DataReader(val_df), batch_size = self.batch_size,
                          num_workers=self.numworker,pin_memory=True,shuffle=False)

    def test_step(self,batch,batch_idx):
        input_ids,attention_mask,label=batch
        out=self(input_ids,attention_mask)
        return {'pred':out,'label':label}
    def test_epoch_end(self, outputs):
        pred=torch.cat([x["pred"] for x in outputs]).detach().cpu().numpy()
        label=torch.cat([x["label"] for x in outputs]).detach().cpu().numpy().ravel()
        self.ground_label=label
        self.predicted_label=pred
        pred=np.argmax(pred,1)
        self.report=classification_report(label,pred)


In [27]:
model=OurModel()
seed_everything(0)
trainer = Trainer(max_epochs=3, 
                  deterministic=True,
                  gpus=1,precision=16,
                  num_sanity_val_steps=0,
#                   accumulate_grad_batches=2,
                  enable_model_summary=False,
                  enable_progress_bar = False,
                  progress_bar_refresh_rate=0
             )
trainer.fit(model)

Global seed set to 0
Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


validation accuracy 0.99
validation accuracy 1.0
validation accuracy 0.99


In [28]:
%%time
trainer.validate(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


validation accuracy 0.99
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.9900000095367432
        val_loss           0.009999999776482582
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
CPU times: user 2.31 s, sys: 1.21 s, total: 3.51 s
Wall time: 3.67 s


[{'val_acc': 0.9900000095367432, 'val_loss': 0.009999999776482582}]

In [29]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{}]

In [30]:
print(model.report)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       240
           1       0.99      1.00      0.99       228

    accuracy                           0.99       468
   macro avg       0.99      0.99      0.99       468
weighted avg       0.99      0.99      0.99       468



In [31]:
torch.save(model.bertmodel.state_dict(), 'bert.pth')

# optimum

In [32]:
#save
token=AutoTokenizer.from_pretrained(model_name,local_files_only=False)
token.save_pretrained("model")
model.bertmodel.save_pretrained("model")


In [33]:
#create optimum onnx
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer


# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained('model', feature="sequence-classification")

##Quantize the model!
quantizer.export(
    onnx_model_path="model.onnx",
    onnx_quantized_model_output_path="model-quantized.onnx",
    quantization_config=qconfig,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/distilbert/modeling_distilbert.py:215: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


PosixPath('model-quantized.onnx')

In [34]:
# testing result
from functools import partial
from datasets import Dataset
from optimum.onnxruntime.model import ORTModel
from optimum.onnxruntime import ORTQuantizer

# Load quantized model
quantizer = ORTQuantizer.from_pretrained('model', feature="sequence-classification")

ort_model = ORTModel("model.onnx", quantizer._onnx_config)

In [40]:
# # %%time


# # Create a dataset or load one from the Hub
# ds = Dataset.from_dict({"sentence": list(val_df.tweet)})
# # Tokenize the inputs
# def preprocess_fn(ex, tokenizer):
#     return tokenizer(ex["sentence"],max_length=64,padding='max_length',truncation=True,)

# tokenized_ds = ds.map(partial(preprocess_fn, tokenizer=quantizer.tokenizer))
# ort_outputs = ort_model.evaluation_loop(tokenized_ds)
# # Extract logits!
# pred=ort_outputs.predictions


In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(val_df.label,np.argmax(pred,1)))

# onnx 

In [36]:
!python -m transformers.onnx --model=model --feature=sequence-classification onnx/

Using framework PyTorch: 1.12.0+cu113
/usr/local/lib/python3.7/dist-packages/transformers/models/distilbert/modeling_distilbert.py:215: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)
Validating ONNX model...
	-[✓] ONNX model output names match reference model ({'logits'})
	- Validating ONNX Model output "logits":
		-[✓] (2, 2) matches (2, 2)
		-[✓] all values close (atol: 1e-05)
All good, model saved at: onnx/model.onnx


In [37]:
from transformers import AutoTokenizer
from onnxruntime import InferenceSession
model_name="distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
session = InferenceSession("onnx/model.onnx")
session.intra_op_num_threads = 12


In [38]:
%%time
tokens=tokenizer(list(val_df.tweet),max_length=64,padding='max_length',truncation=True, return_tensors="np")
outputs = session.run(output_names=["logits"], input_feed=dict(tokens))

CPU times: user 38.5 s, sys: 316 ms, total: 38.8 s
Wall time: 38.6 s


In [39]:
print(classification_report(val_df.label,np.argmax(outputs[0],1)))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       240
           1       0.99      1.00      0.99       228

    accuracy                           0.99       468
   macro avg       0.99      0.99      0.99       468
weighted avg       0.99      0.99      0.99       468

